# Amazon Fine Food Reviews Analysis

Data source : https://www.kaggle.com/snap/amazon-fine-food-reviews

The Amazon Fine Food Reviews dataset consists of reviews of fine foods from Amazon.

Number of reviews: 568,454
Number of users: 256,059
Number of Products: 74,258
Timespan: Oct 1999 - Oct 2012
Number of Attributes/Columns in data: 10

Attribute Information:
1. Id
2. ProductId - unique identifier for the product
3. UserId - unique identifier for the user
4. ProfileName
5. HelpfulnessNumerator - number of users who found the review helpful
6. HelpfulnessDenominator - number of users who indicated whether they found the review helpful or not
7. Score - rating between 1 and 5
8. Time - timestamp for the review
9. Summary - brief summary of the review
10. Text - text of the review

#### Objective

Given a review, determine whether the review is positive (Rating of 4 or 5) or negative (rating of 1 or 2).

[Q] How to determine if a review is positive or negative?

[Ans] We could use the Score/Rating. A rating of 4 or 5 could be considered a positive review. A review of 1 or 2 
could be considered negative. A review of 3 is neutral and therefore ignored. Note that this is an approximate and proxy way of determining the polarity (positivity/negativity) of a review.

## Loading the dataset

The dataset is available in two forms
1. .csv file
2. SQLite database

In order to load the data, We have used the SQLITE dataset as it is easier to query the data and visualise the data effectively.

Here as we only want to get the global sentiment of the recommendations (positive or negative), we will purposefully ignore all Scores equal to 3.

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import sqlite3

In [21]:
# Using the SQLite Table to read data.
con = sqlite3.connect('database.sqlite')

# Filtering only positive and negative reviews i.e. not taking
# into consideration those reviews whose score = 3
filtered_data = pd.read_sql_query('select * from reviews where score != 3 limit 100000', con)

print('Number of datapoints in our data {}'.format(filtered_data.shape))
filtered_data.head()

Number of datapoints in our data (100000, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [22]:
# Give reviews with a score > 3 a positive rating,
# and reviews with a score < 3 a negative rating
def partition(x):
    if x < 3:
        return 0 # negative
    return 1 # positive

actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition)
filtered_data['Score'] = positiveNegative
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,0,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,1,1350777600,Great taffy,Great taffy at a great price. There was a wid...


## Exploratory Data Analysis

### Data Cleaning : Deduplication

It is observed (as shown in the table below) that the reviews data had many duplicate entries. Hence it was necessary to remove duplicates in order to get unbiased results for the analysis of the data.

Following is an example:

In [23]:
filtered_data[filtered_data['UserId'] == 'AR5J8UI46CURR']

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
67901,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,1,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
72060,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,1,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


- As can be seen above the same user has multiple reviews with the same values for HelpfulnessNumerator, HelpfulnessDenominator, Score, Time, Summary and Text and on doing analysis it was found that 

  ProductId=B000HDOPZG was Loacker Quadratini Vanilla Wafer Cookies, 8.82-Ounce Packages (Pack of 8)

  ProductId=B000HDL1RQ was Loacker Quadratini Lemon Wafer Cookies, 8.82-Ounce Packages (Pack of 8) and so on
  

- It was inferred after analysis that reviews with same parameters other than ProductId belonged to the same product just having different flavour or quantity. Hence in order to reduce redundency it was decided to eliminate the rows having same parameters.


- The method used to eliminate was that we first sort the data according to ProductId and then just keep the first similar product review and delete the others. For eg. in the above just the review for ProductId=B000HDL1RQ remains. This method ensures that there is only one representative for each product and deduplication without sorting would lead to possibility of different representatives still existing for the same product.

In [24]:
# Sorting data according to ProductId in ascending order
sorted_data = filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [25]:
# Deduplication of entries
final = filtered_data.drop_duplicates(subset={"UserId", "ProfileName", "Time", "Text"}, keep='first', inplace=False)
final.shape

(87775, 10)

In [26]:
# Checking to see how much % of data still remains
(final.size / filtered_data.size) * 100

87.775

<b>Observation : </b>It was also seen that in two rows given below the value of HelpfulnessNumerator is greater than HelpfulnessDenominator which is not practically possible hence these two rows too are removed from calculations.

In [27]:
final[final['HelpfulnessNumerator'] > final['HelpfulnessDenominator']]

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
41159,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,1,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...
59301,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,1,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...


In [28]:
final = final[final['HelpfulnessNumerator'] <= final['HelpfulnessDenominator']]

In [29]:
# Before starting the next phase of preprocessing lets see the number of entries left
print(final.shape)

# How many positive and negative reviews are present in our dataset ?
final['Score'].value_counts()

(87773, 10)


1    73592
0    14181
Name: Score, dtype: int64

## Text Preprocessing: Stemming, stop-word removal and Lemmatization

Now that we have finished deduplication our data requires some preprocessing before we go on further with analysis
and making the prediction model.

Hence in Preprocessing phase we do following in the order below:-

1. Begin by removing html tags
2. Remove any punctuations or limited set of special characters like , or . or # etc.
3. Check if the word is made up of english letters and is not alpha-numeric.
4. Check to see if the length of the word is greater than 2 (as it was researched that there is no adjective in 2-letters)
5. Convert the word to lowercase.
6. Remove Stopwords.
7. Finally Snowball stemming the word (it was observed to be better than Porter Stemming)

After which we collect the words used to describe positive and negative reviews.

In [30]:
# finding sentences containing HTML tags
import re
i = 0
for sent in final['Text'].values:
    if len(re.findall('<.*?>', sent)) > 0:
        print(i)
        print(sent)
        break
    i += 1

10
I don't know if it's the cactus or the tequila or just the unique combination of ingredients, but the flavour of this hot sauce makes it one of a kind!  We picked up a bottle once on a trip we were on and brought it back home with us and were totally blown away!  When we realized that we simply couldn't find it anywhere in our city we were bummed.<br /><br />Now, because of the magic of the internet, we have a case of the sauce and are ecstatic because of it.<br /><br />If you love hot sauce..I mean really love hot sauce, but don't want a sauce that tastelessly burns your throat, grab a bottle of Tequila Picante Gourmet de Inclan.  Just realize that once you taste it, you will never want to use any other sauce.<br /><br />Thank you for the personal, incredible service!


In [31]:
# defining some helper functions for text preprocessing
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

stop = set(stopwords.words('english')) # set of stopwords
sno = SnowballStemmer('english') # initialising the snowball stemmer

def cleanhtml(sentence): # function to clean html tags from sentence
    tag = re.compile('<.*?>')
    clean_sentence = re.sub(tag, ' ', sentence)
    return clean_sentence

def cleanpunc(sentence): # function to clean the sentence from any punctuation or special characters
    clean_sentence = re.sub(r'[?|!|\'|"|#]', r'', sentence)
    clean_sentence = re.sub(r'[.|,|)|(|\|/]', r' ', clean_sentence)
    return clean_sentence

In [32]:
print(stop)
print('***************************************************')
print(sno.stem('tasty'))

{'himself', "wasn't", "you'll", 'any', 'm', "mightn't", 'nor', 'that', 'me', 'herself', 'mustn', 'more', 'down', 'wasn', 'd', 'wouldn', 'yourself', 'them', 'shouldn', 'from', "aren't", "hasn't", 'ain', 'has', 'doing', 'isn', 'yours', 'themselves', 'ma', 'they', 'those', 'why', 'before', 'mightn', "haven't", 'it', 'been', "that'll", 'theirs', 'weren', "you've", 'whom', 'then', 'couldn', 'him', 'is', 'an', 'hers', 'over', "don't", 'hasn', 'into', 'were', 'shan', 'all', 'about', "couldn't", 'until', 'his', 't', "needn't", 'other', "won't", 'do', "doesn't", 'hadn', 'am', 're', 'out', 'now', 'be', 've', 'their', 'being', 'above', 'she', 'having', 'which', 'up', 'ourselves', "you're", 'aren', 'itself', "weren't", 'our', 'doesn', 'your', "shan't", 'will', 'most', 'll', 'can', 'here', "should've", 'so', 'in', 'ours', 'to', 'too', 'where', 'each', 'you', 'just', 'o', 'didn', 'yourselves', "you'd", 'i', 'below', 'some', 'than', 'very', 'have', 'the', 'who', 'her', 'he', 'what', 'under', 'this', 

In [38]:
# Code for implementing step-by-step the checks mentioned in the pre-processing phase
# The code takes a while to run as it needs to run on 100k sentences.
import os
from tqdm import tqdm
if not os.path.isfile('final.sqlite'):
    i = 0
    cleaned_reviews = []
    all_positive_words = [] # store words from positive reviews here
    all_negative_words = [] # store words from negative reviews here
    for sent in tqdm(final['Text'].values):
        cleaned_words = [] # storing each cleaned word from sent
        sent = cleanhtml(sent) # remove html tags
        for w in sent.split():
            for cleaned_word in cleanpunc(w).split():
                if cleaned_word.isalpha() and len(cleaned_word)>2:
                    if cleaned_word.lower() not in stop: # stopword removal
                        s = (sno.stem(cleaned_word.lower())).encode('utf8') # stemming
                        cleaned_words.append(s)
                        if final['Score'].values[i] == 1:
                            all_positive_words.append(s) # list of all words used to describe positive reviews
                        if final['Score'].values[i] == 0:
                            all_negative_words.append(s) # list of all words used to describe negative reviews
                    else:
                        continue
                else:
                    continue
        str1 = b" ".join(cleaned_words) # final string of cleaned words
        cleaned_reviews.append(str1)
        i += 1

100%|██████████| 87773/87773 [01:46<00:00, 822.43it/s] 


In [39]:
import pickle
if not os.path.isfile('final.sqlite'):
    ############-------- storing the preprocessed data into .sqlite file --------#############
    final['CleanedText'] = cleaned_reviews # adding a column of CleanedText which displays the data after pre-processing of the review
    final['CleanedText'] = final['CleanedText'].str.decode('utf-8')
    # store final table into a SQLite table for future.
    conn = sqlite3.connect('final.sqlite')
    c = conn.cursor()
    conn.text_factory = str
    final.to_sql('Reviews', conn, if_exists='replace')
    conn.close()
    
    with open('positive_words.pkl', 'wb') as f:
        pickle.dump(all_positive_words, f)
    with open('negative_words.pkl', 'wb') as f:
        pickle.dump(all_negative_words, f)